# \`\-.\_ numPy \_.\-\`
The aim for this notebook is to familiarize myself with the __numPy library__ a little more.

In [103]:
from __future__ import annotations
from pprint import pprint
from typing import Iterator, Iterable
import random
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

#### numPy is all about vectorization.
Below is a random walker example using:
1. An object oriented approach
2. Procedural approach
3. Vectorized approach

#### 1. OO

In [5]:
class RandomWalker:
    def __init__(self) -> RandomWalker:
        self.position: int = 0
            
    def walk(self, n: int) -> Iterator[int]:
        self.position = 0
        for i in range(n):
            yield self.position
            # randomly adds 1 or -1 to self.position
            self.position += 2 * random.randint(0, 1) - 1

In [9]:
walker = RandomWalker()
%timeit for position in walker.walk(n=10000): position

7.08 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 2. Proc

In [10]:
def random_walk(n):
    position: int = 0
    walk = [position]
    for i in range(n):
        position += 2 * random.randint(0, 1) - 1
        walk.append(position)
    return walk

walk = random_walk(1000)

In [11]:
# less time probably due to the inner Python OO machinery
%timeit random_walk(n=10000)

6.65 ms ± 95.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 3. Vect

In [15]:
# creating iterators for efficient looping
# rewrite the function by first generating all the steps
# and accumulate them without any loop

def random_walk_faster(n=1000):
    from itertools import accumulate
    steps: list[int] = random.choices([-1, +1], k=n)
    return [0] + list(accumulate(steps))

walk = random_walk_faster(1000)

In [16]:
%timeit random_walk_faster(n=10000)

956 µs ± 3.95 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Instead of looping for picking sequential steps and add them to the current position, we first generated all the steps at once and used the `accumulate` function to compute all the positions.

### Anatomy of an array
When we want to clear all the values from an array which has dtype `np.float32`, how do we write it to maximize speed?

In [27]:
Z = np.ones(4*1000000, np.float32)
Z[...] = 0

In [32]:
# size -> number of items in array
# itemsize -> length of one array item in bytes
Z.size, Z.itemsize

(4000000, 4)

`a.view(some_dtype)` or `a.view(dtype=some_dtype)` constructs a view of the array’s memory with a different data-type. This can cause a reinterpretation of the bytes of memory.

`Z.size * Z.itemsize` can be divided by the new dtype itemsize, due to the array able to be casted into other "compatible" data types.

In [31]:
dash = lambda: print("-"*60)

print("float16")
%timeit Z.view(np.float16)[...] = 0
dash(); print("int16")
%timeit Z.view(np.int16)[...] = 0
dash(); print("int32")
%timeit Z.view(np.int32)[...] = 0
dash(); print("float32")
%timeit Z.view(np.float32)[...] = 0
dash(); print("int64")
%timeit Z.view(np.int64)[...] = 0
dash(); print("float64")
%timeit Z.view(np.float64)[...] = 0
dash(); print("complex128")
%timeit Z.view(np.complex128)[...] = 0
dash(); print("int8")
%timeit Z.view(np.int8)[...] = 0

float16
1.87 ms ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
------------------------------------------------------------
int16
1.91 ms ± 40.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
------------------------------------------------------------
int32
2.11 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
------------------------------------------------------------
float32
1.98 ms ± 52.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
------------------------------------------------------------
int64
1.91 ms ± 51.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
------------------------------------------------------------
float64
1.92 ms ± 38.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
------------------------------------------------------------
complex128
1.92 ms ± 23.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
------------------------------------------------------------
int8
1.77 ms

#### Memory layout
An instance of class ndarray consists of a contiguous one-dimensional segment of computer memory (owned by the array, or by some other object), combined with an indexing scheme that maps N integers into the location of an item in the block.

In [40]:
Z = np.arange(9).reshape(3, 3).astype(np.int16)
print(Z)
print('\nitemsize:', Z.itemsize,
      '\nshape:', Z.shape,
      '\nndim:', Z.ndim)

[[0 1 2]
 [3 4 5]
 [6 7 8]]

itemsize: 2 
shape: (3, 3) 
ndim: 2


In [47]:
# strides of the array define the number of bytes to step
# in each dimension when traversing the array
strides = Z.shape[1] * Z.itemsize, Z.itemsize
print('(by row, by column)')
print(strides) # also exists as Z.strides

(by row, by column)
(6, 2)


In [50]:
# how to manually index an element in memory
index = 1, 1 # location of element as index
print(Z[index].tobytes())

offset = 0 # start at 0
for i in range(Z.ndim): # 2 dim, so [0, 1] where 0 -> row and 1 -> col
    offset_start += Z.strides[i]*index[i]
offset_end = offset_start + Z.itemsize

print(Z.tobytes()[offset_start:offset_end]) # nice

b'\x04\x00'
b'\x08\x00'


In [54]:
# if we take a slize of Z, the result is a view of the
# base array Z
V = Z[::2, ::2]
print(Z)
print(V)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 2]
 [6 8]]


#### Views and copies
Must distinguish between indexing and fancy indexing,
the first will return a view while the second will return a copy.
The difference is important, because modifying the view modifies the base array, while this is not true for a copy.

In [56]:
Z = np.zeros(9)
print("old:")
print(Z)
Z_view = Z[:3]
Z_view[...] = 1
print("view modified:")
print(Z)

old:
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
view modified:
[1. 1. 1. 0. 0. 0. 0. 0. 0.]


In [58]:
Z = np.zeros(9)
print("old:")
print(Z)
Z_copy = Z[[0, 1, 2]] # individual indices
# shape of result reflects the shape of the index arrays
# rather than the shape of the array being indexed
Z_copy[...] = 1
print("after modifying copy:")
print(Z) # doesn't change

old:
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
after modifying copy:
[0. 0. 0. 0. 0. 0. 0. 0. 0.]


If you need fancy indexing, it's better to keep a copy of your fancy index (especially if it was complex to compute) and work with it.

Examples:

In [60]:
X = np.arange(12).reshape(3, 4)
X

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [62]:
row = np.array([0, 1, 2])
col = np.array([2, 1, 3])
X[row, col] # [0, 2] -> 2, which means 0th row, 2nd col

array([ 2,  5, 11])

#### Bried aside: broadcasting
For arrays of the same size, binary operations are performed on an element-by-element basis:

In [64]:
a = np.array([0, 1, 2])
b = np.array([5, 5, 5])
a + b

array([5, 6, 7])

In [65]:
# just as easily add a scalar
a + 5

array([5, 6, 7])

In [69]:
# when we add a one dimensional array to a 2-dim one...
M = np.ones((3, 3))
M

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [71]:
M + a # a is stretched in order to match the shape of M

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

In [74]:
# consider this example:
a = np.arange(3)
b = np.arange(3)[:, np.newaxis]

print(a)
print(b)

[0 1 2]
[[0]
 [1]
 [2]]


In [76]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

#### What is newaxis??

`np.newaxis` is used to increase the dimension of the array by one more dimension, when used *once*. Thus,
- 1D array -> 2D array
- 2D array -> 3D array

and so on...

In [79]:
A = np.array([2, 0, 1, 8])
print(A, A.shape)

[2 0 1 8] (4,)


In [84]:
A1 = A[np.newaxis, :] # (4,) -> (1, 4)
print(A1, A1.shape) 

[[2 0 1 8]] (1, 4)


In [85]:
A2 = A[:, np.newaxis] # (4,) -> (4, 1)
print(A2, A2.shape)

[[2]
 [0]
 [1]
 [8]] (4, 1)


Comes in handy when you want to explicitly convert a 1D array to either a row vector or column vector.

In [86]:
X

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [87]:
row, col

(array([0, 1, 2]), array([2, 1, 3]))

In [89]:
# each row value is matched with each column vector
X[row[:, np.newaxis], col]

array([[ 2,  1,  3],
       [ 6,  5,  7],
       [10,  9, 11]])

In [111]:
# indices like so:
np.array([[int(str(r) + str(c))*.1 for c in col] for r in row])

array([[0.2, 0.1, 0.3],
       [1.2, 1.1, 1.3],
       [2.2, 2.1, 2.3]])